# RUSCORPORA BUA PARSER

* Подготовили - Анастасия Костяницына, Артем Копецкий

* Парсинг параллельного бурятского корпуса. 

In [2]:
import pickle
from collections import defaultdict

from lxml import etree
from string import punctuation

Структура данных для примеров из НКРЯ (text_bua - бурятские примеры, text_rus - русские примеры)

In [3]:
class Pair:
    
    def __init__(self, id_, bua, rus):
        self.id_ = id_
        self.tokens_bua, self.infos_bua = bua
        self.tokens_rus, self.infos_rus = rus
        
        self.text_bua = ''.join(self.tokens_bua).strip()
        self.text_rus = ''.join(self.tokens_rus).strip()
        
    def find_rus_in_bua(self, word):
        res = []
        
        for token, info in zip(self.tokens_bua, self.infos_bua):
            _info = info.get('sem')
            
            if _info is not None and word in _info:
                res.append((word, token, info))
                
        return res

Скачиваем примеры

In [4]:
def get_infos_dict(word):
    return {info: [x.text.strip(punctuation) for x in word.xpath(XPATH_TO_INFO % info)] for info in INFOS}


def parse_doc(doc):
    tokens = []
    infos = []
    
    
    for el in doc.getchildren():
     
        if el.tag == 'text':
            tokens.append(el.text)
            infos.append({})
            
        else:
            tokens.append(el.attrib['text'])
            infos.append(get_infos_dict(el))
            
    return tokens, infos


def parse_page(url):
    
    tree = etree.parse(url)
    root = tree.getroot()
    
    all_ = []
    
    for doc1 in root.xpath(XPATH_TO_DOC):
        
        d = {}
        pairs_parsed = []
        
        for child in doc1:
            
            if child.tag == 'attributes':
                for i in child:
                    d[i.attrib['name']] = i.attrib['value']
            
            if child.tag == 'para':
                
                doc1 = child[0]
                doc2 = child[1]
        
                if doc1.attrib['sid'] == doc2.attrib['sid']:
                    if doc1.attrib['language'] == 'bua':
                        pairs_parsed.append(Pair(doc1.attrib['sid'], parse_doc(doc1), parse_doc(doc2)))
                    else:
                        pairs_parsed.append(Pair(doc1.attrib['sid'], parse_doc(doc2), parse_doc(doc1)))
               
        #d['pairs'] = pairs_parsed
        all_.append((d, pairs_parsed))
        
    return all_

In [1]:
XPATH_TO_DOC = 'searchresult/body/result/document'
XPATH_TO_PAIRS = 'searchresult/body/result/document/para' 
XPATH_TO_INFO = 'ana/el[@name="%s"]/el-group/el-atom'
INFOS = ('lex', 'gramm', 'sem', 'flags')

In [3]:
url_template = 'http://search1.ruscorpora.ru/dump.xml?mysent=&mysize=273016&mysentsize=6674&dpp=&spp=&spd=&text=lexgramm&mode=para&sort=gr_tagging&env=alpha&mycorp=%28lang%3A"bua"+%7C+lang_trans%3A"bua"%29&parent1=0&level1=0&lex1={}&gramm1=&flags1=&sem1=&parent2=0&level2=0&min1=1&max1=1&lex2=&gramm2=&flags2=&sem2='

Грамм. запрос "отец"

In [4]:
url = url_template.format('отец')

In [7]:
%%time
x = parse_page(url)

CPU times: user 156 ms, sys: 15.6 ms, total: 172 ms
Wall time: 517 ms


In [5]:
url

'http://search1.ruscorpora.ru/dump.xml?mysent=&mysize=273016&mysentsize=6674&dpp=&spp=&spd=&text=lexgramm&mode=para&sort=gr_tagging&env=alpha&mycorp=%28lang%3A"bua"+%7C+lang_trans%3A"bua"%29&parent1=0&level1=0&lex1=отец&gramm1=&flags1=&sem1=&parent2=0&level2=0&min1=1&max1=1&lex2=&gramm2=&flags2=&sem2='

# Основная часть

In [11]:
with open('bua_words.txt', 'r') as f:
    WORDS = [x.rstrip('\n') for x in f.readlines()]

In [15]:
from tqdm import tqdm_notebook
from time import sleep
from random import randint, random

In [32]:
WORDS_PAIRS = {}

In [36]:
for i, w in enumerate(tqdm_notebook(WORDS)):
    url = url_template.format(w)
    
    try:
        WORDS_PAIRS[w] = parse_page(url)
    except:
        continue
    
    if random() < 0.2:
        sleep(1)

In [39]:
def get_match_status(match):
    """
    match: i-th el of `.find_rus_in_bua` output
    """
    
    infos = match[-1]
    
    lex = infos.get('lex') or []
    
    # deny if more than one bua lexeme 
    if len(lex) > 1:
        return 2
    
    gramm = infos.get('gramm') or []
    
    # mark if more than one gramm info
    if len([_ for x in gramm if x.isupper()]) > 1:
        return 1
    
    return 0


def get_matches(words_pairs):
    matches = defaultdict(list)
    
    for word, data in words_pairs.items():
        for doc, pairs in data:
            for pair in pairs:
                c_matches = pair.find_rus_in_bua(word)
                matches[word].extend(
                    [{'status': get_match_status(m), 'doc': doc, 'pair': pair, 'match': m[1:]}
                     for m in c_matches
                    ]
                )
                
    return matches

In [40]:
%%time
matches = get_matches(WORDS_PAIRS)

CPU times: user 688 ms, sys: 281 ms, total: 969 ms
Wall time: 984 ms


In [16]:
with open('bua_rus_matches.pkl', 'wb') as f:
    pickle.dump(matches, f)

In [7]:
with open('bua_rus_matches.pkl', 'rb') as f:
    matches = pickle.load(f)

Структура данных для словарной статьи

In [42]:
class Entry:
    
    def __init__(self, lex):
        self.lex = lex
        
        self.pos = {}
        self.sem = {}
        self.gramm = {}
        self.examples = defaultdict(list)
        
    def add_pos(self, pos):
        if pos not in self.pos:
            self.pos[pos] = {}
    
    def add_sem(self, pos, sem):
        if sem not in self.pos[pos]:
            self.pos[pos][sem] = {}
    
    def add_form(self, pos, sem, gramm, form):
        if gramm not in self.pos[pos][sem]:
            self.pos[pos][sem][gramm] = form
            
    def add_example(self, sem, doc, pair):
        self.examples[sem].append((doc, pair))
            
    def get_tree(self):
        return self.pos

    

Достаем из примеров из НКРЯ сочетания слов и грамматической информации

In [43]:
def get_all_gramms(gramm_info):
    res = []
    c_info = gramm_info[:1]

    for x in gramm_info[1:]:
        if not x.isupper():
            c_info.append(x)

        else:
            res.append(c_info)
            c_info = [x]

    res.append(c_info)
        
    return res


def build_entries_from_matches(matches):
    entries = {}
    
    for word, c_matches in matches.items():
        c_entries = {}
        
        for match in c_matches:
            c_status = match['status']
            
            # ambiguous case: multiple lexemes
            if c_status == 2:
                continue
                
            form, info = match['match']
            lex = info['lex'][0]
            
            if lex not in c_entries:
                c_entries[lex] = Entry(lex)

            entry = c_entries[lex]
            
            # unambiguous case: single lexeme, single gramm
            if c_status == 0:
                pos = info['gramm'][0]
                gramm = ', '.join(info['gramm'][1:])
                sem = ', '.join(info['sem'])

                entry.add_pos(pos)
                entry.add_sem(pos, sem)
                entry.add_form(pos, sem, gramm, form.lower())
                entry.add_example(sem, match['doc'], match['pair'])
            
            # semi-ambiguous case: single lexeme, multiple gramms
            else:
                gramms = get_all_gramms(info['gramm'])
                sems = [', '.join(info['sem'])] + ['AMBIGUOUS'] * (len(gramms) - 1)
                
                for gramm, sem in zip(gramms, sems):
                    pos = gramm[0]
                    _gramm = ', '.join(gramm[1:])
                    
                    entry.add_pos(pos)
                    entry.add_sem(pos, sem)
                    entry.add_form(pos, sem, _gramm, form.lower())
                    entry.add_example(sem, match['doc'], match['pair'])
                    
        entries[word] = c_entries
        
    return entries
        
    
    

In [44]:
%%time
entries = build_entries_from_matches(matches)

CPU times: user 31.2 ms, sys: 15.6 ms, total: 46.9 ms
Wall time: 27.9 ms


In [47]:
with open('bua_rus_entries.pkl', 'wb') as f:
    pickle.dump(entries, f)

In [ ]:
with open('bua_rus_entries.pkl', 'rb') as f:
    entries = pickle.load(f)

### Примеры:

Ищем все вхождения Отца в нашей базе данных.

In [203]:
b = []
for i in docs:
    pairs = i[1]
    for pair in pairs:
        a = pair.find_rus_in_bua('отец')
        b += a
        print(a)
        print(pair.text_bua)
        print(pair.text_rus)
        print()
print(len(b))

[('отец', 'Эсэгынгээ', {'lex': ['эсэгэ'], 'gramm': ['S', 'gen', 'poss'], 'sem': ['отец'], 'flags': ['bua', 'capital', 'first', 'nacc', 'posred']})]
Эсэгынгээ хэрэгээр Үдын байшан ошоһон аад, орой гэртээ хүрэжэ ерээ һэм, — гэжэ Бадан мүн лэ илдамаар харюусаад, урдаһаань баһа нэнгэжэ, ургынгаа гуйба мүртөө шагтагалһан баруун гараараа морин дээрэһээ Залмые хүзүүдэн, даараһандаа улайхадаһан хасартань өөрынгөө хасар няаба.
— Отец отправил в Удинский острог, вернулся поздно. Не сердись, если не острог, обязательно приехал бы.

[('отец', 'эсэгэ', {'lex': ['эсэгэ'], 'gramm': ['S', 'acc', 'poss', 'S', 'nom'], 'sem': ['отец'], 'flags': ['bua', 'casered', 'nacc', 'posred']})]
— Манай адуун.    Минии эсэгэ Жарантай хоёрой.    — Аа, мүнөө эдэ адуунтнай манай болоо.
— Наш.    Моего отца и Жарантая,    — Теперь это наш табун.

[]
— Тураахиин хүбүүн Бадан гэжэ аабза.    Тэрэшни эсэгэдээл адли ород эльгэтэй, саашаа һанаатай амитан байха.    Тэрэ Тураахиһаа боложо, буряадууд манда алба татабари түлэхэеэ

Пары бур. и рус. предложений и вся сопутствующая информация возвращаются как список типов `Pair`

In [8]:
len(pairs)

30

In [9]:
pair = pairs[0]

Полный текст хранится в атрибутах `text_bua` / `text_rus`

In [10]:
print(f'{pair.text_bua}\n{pair.text_rus}') 

Эсэгынгээ хэрэгээр Үдын байшан ошоһон аад, орой гэртээ хүрэжэ ерээ һэм, — гэжэ Бадан мүн лэ илдамаар харюусаад, урдаһаань баһа нэнгэжэ, ургынгаа гуйба мүртөө шагтагалһан баруун гараараа морин дээрэһээ Залмые хүзүүдэн, даараһандаа улайхадаһан хасартань өөрынгөө хасар няаба.
— Отец отправил в Удинский острог, вернулся поздно. Не сердись, если не острог, обязательно приехал бы.


Сами токены и их информация хранятся в `tokens_язык` и `infos_язык`

In [11]:
for token, info in zip(pair.tokens_bua, pair.infos_bua):
    if info:
        print(f'token: {token}\ninfo: {info}\n')

token: Эсэгынгээ
info: {'lex': ['эсэгэ'], 'gramm': ['S', 'gen', 'poss'], 'sem': ['отец'], 'flags': ['bua', 'capital', 'first', 'nacc', 'posred']}

token: хэрэгээр
info: {'lex': ['хэрэг'], 'gramm': ['S', 'ins'], 'sem': ['дело'], 'flags': ['bua', 'nacc', 'posred']}

token: Үдын
info: {'lex': ['үдын', 'үдэ'], 'gramm': ['A', 'S', 'gen'], 'sem': ['поддень', 'полдневный', 'полуденный'], 'flags': ['bua', 'nacc', 'posred']}

token: байшан
info: {'lex': ['байшан', 'байшаха'], 'gramm': ['S', 'nom', 'V', 'circumst', 'cvb'], 'sem': ['байха_очутиться_где-л', 'дом', 'изба'], 'flags': ['bua', 'nacc', 'posred']}

token: ошоһон
info: {'lex': ['ошохо'], 'gramm': ['V', 'partcp', 'praet', '3p'], 'sem': ['идти_куда-л'], 'flags': ['bua', 'nacc', 'posred']}

token: аад
info: {'lex': ['аад'], 'gramm': ['A'], 'sem': ['но'], 'flags': ['bcomma', 'bmark', 'bua', 'nacc']}

token: орой
info: {'lex': ['орой', 'оройхо'], 'gramm': ['S', 'nom', 'V', 'imper', '2p', 'sg'], 'sem': ['вовсе', 'совсем'], 'flags': ['acomma', 

Наконец, поиск бурятского токена с определенным переводом на русский через метод `find_rus_in_bua`  
Возвращается список всех соответствующих бурятских токенов и их разборов

Ищем отца

In [12]:
pair.find_rus_in_bua('отец')

[('отец',
  'Эсэгынгээ',
  {'lex': ['эсэгэ'],
   'gramm': ['S', 'gen', 'poss'],
   'sem': ['отец'],
   'flags': ['bua', 'capital', 'first', 'nacc', 'posred']})]

Ищем коня)

In [13]:
pair.find_rus_in_bua('конь')

[('конь',
  'морин',
  {'lex': ['морин'],
   'gramm': ['S', 'nom'],
   'sem': ['конь', 'лошадь', 'мерин'],
   'flags': ['bua', 'nacc', 'posred']})]

## Сложные случаи

В данных есть сложные случаи, когда словоформа соответсвует сразу нескольким леммам, при этом у нее соответвенно несколько вариантов грам. разбора и несолько значений, которые сложно соотнести. Пример: 

* ('я', 'бидэ', {'lex': ['бидэ', 'бидэхэ', 'би'], 'gramm': ['S', 'acc', 'poss', 'S', 'nom', 'V', 'imper', '2p', 'sg', 'S', 'dat'], 'sem': ['мы', 'путешествовать', 'странствовать', 'я'], 'flags': ['bua', 'nacc', 'posred']})

Тут получается 3 леммы, 4 грам. разбора и "4" значения (по сути 3, потому что  'путешествовать' и 'странствовать' близкие). Не понятно, как делить. Чтобы избежать ошибок, я взяла случаи, когда словоформе соответвует одна лемма, чтобы точно быть уверенными в разборе.

 
* ходо = ['вы', 'насквозь', 'пере', 'с', 'также_соответствует_приставкам_про', 'через_что-л'], 'gramm': ['ADV'], 'ex': [183]
(разные значения, вообще надо бы делить на sense1, sense2, но как тогда разделить примеры и грам инфу? в остальных случаях значения однотипны, поэтому они объединяются в одно значение)
* мэдэхэ = ['знать', 'узнавать_о_чем-л'], 'gramm': ['V', 'partcp', 'fut', '3p', 'sg'] (вроде лемма, но форма указанна как не инф.)
* гурбан = ['три', 'трое', 'тройка'], 'gramm': ['NUM', 'S', 'nom']